# More On The Duckdb Python API
### This notebook gives a little more info on using python functions with duckdb
### For example take this function which gets json from a url

In [169]:
import json
from typing import List, Union

from duckdb.typing import DuckDBPyType
import pandas as pd
import requests


def get_url_json(url: str, keys: list[str]) -> DuckDBPyType(list[dict[str,str]]):
    out_dict = requests.get(url).json()
    for key in keys:
        out_dict = out_dict[key]
    for item in out_dict:
        for key in item:
            try:
                item[key] = str(item[key])
            except Exception:
                item[key] = '' 
    return out_dict


### Verifying that it works:

In [171]:
get_url_json('http://dummyjson.com/users', ['users'])[0]

{'id': '1',
 'firstName': 'Emily',
 'lastName': 'Johnson',
 'maidenName': 'Smith',
 'age': '28',
 'gender': 'female',
 'email': 'emily.johnson@x.dummyjson.com',
 'phone': '+81 965-431-3024',
 'username': 'emilys',
 'password': 'emilyspass',
 'birthDate': '1996-5-30',
 'image': 'https://dummyjson.com/icon/emilys/128',
 'bloodGroup': 'O-',
 'height': '193.24',
 'weight': '63.16',
 'eyeColor': 'Green',
 'hair': "{'color': 'Brown', 'type': 'Curly'}",
 'ip': '42.48.100.32',
 'address': "{'address': '626 Main Street', 'city': 'Phoenix', 'state': 'Mississippi', 'stateCode': 'MS', 'postalCode': '29112', 'coordinates': {'lat': -77.16213, 'lng': -92.084824}, 'country': 'United States'}",
 'macAddress': '47:fa:41:18:ec:eb',
 'university': 'University of Wisconsin--Madison',
 'bank': "{'cardExpire': '03/26', 'cardNumber': '9289760655481815', 'cardType': 'Elo', 'currency': 'CNY', 'iban': 'YPUXISOBI7TTHPK2BR3HAIXL'}",
 'company': "{'department': 'Engineering', 'name': 'Dooley, Kozey and Cronin', 'ti

### Now with a function that is type annotated we can pass it to duckdb using the function API, which requires a name to call the function and the function.
### We can do some pretty wild stuff with this including this example where we get data from an API and use it to run SQL against it.

In [178]:
import duckdb

conn = duckdb.connect()
conn.create_function("get_url_json", get_url_json)
conn.sql("""
WITH base AS (
    SELECT unnest(get_url_json('http://dummyjson.com/users', ['users'])) AS data
),
output AS (
    SELECT 
         json_extract_string(data, '$.firstName') AS first_name,
         json_extract_string(data, '$.lastName') AS last_name
    FROM base
)
SELECT * FROM output
LIMIT 5
""").df()

,first_name,last_name
0,Emily,Johnson
1,Michael,Williams
2,Sophia,Brown
3,James,Davis
4,Emma,Miller


### One note is that duckdb has a nice feature in that you can pass dataframes in local memory and run sql against them.
### For example here is the same idea, but with generating a dataframe and then passing it into the sql query with duckdb.

In [177]:
df = pd.DataFrame(get_url_json('http://dummyjson.com/users', ['users']))

duckdb.sql("""
SELECT
    firstName AS first_name,
    lastName AS last_name
FROM df
LIMIT 5
""").df()

,first_name,last_name
0,Emily,Johnson
1,Michael,Williams
2,Sophia,Brown
3,James,Davis
4,Emma,Miller


### For further reading I recommend looking at the duckdb docs on the function API [here](https://duckdb.org/docs/api/python/function.html)